In [1]:
import json
import numpy as np
import pandas as pd
import pickle

with open('./python100.json') as data_file:
    all_data = [json.loads(r) for r in data_file.readlines()]

In [2]:
len(all_data)

100

In [3]:
# input: a code source with multiple functions
# output: a dictionary: key is the code index, value is all functions and their children idx 
# e.g. full_list[index] =  (f_info,[children_idx])

def find_function_children(data):

    functions = [(i,text) for i, text in enumerate(data) if text['type'] == 'FunctionDef']

    full_list = {}
    for (index, f) in functions:
        ans = []
        row = f['children']
        while row:
            for i in row:
                ans.append(i)
            row = [node for root in row for node in data[root].get('children',[]) if node]
        full_list[index] =  (f,ans)
    
    return full_list

In [13]:
def check_child(type_name, data):
    if data['type'] == type_name:
        if data['children']:
            return data['children']

In [14]:
# input: k,v in full_list,  v[1]
# output: a string with all functions call e.g. 'create_stubs,first,AndReturn,ReplayAll,vpnservice_get,request'

def find_call_func(function_list, data):
    
    ans = []
    
    for children_id in function_list:
        if check_child('Call', data[children_id]):
            for j in data[children_id]['children']:
                if check_child('AttributeLoad', data[j]):
                    for z in check_child('AttributeLoad', data[j]):
                        if data[z]['type'] == 'attr': 
                            ans.append(data[z]['value'])
    call_func = ','.join(ans)
    return call_func
    

In [5]:
# input: k,v in full_list,  v[1]
# output: a string, the function's docstring 
def find_docstring(function_list, data):
    
    docstring = ''
    
    for children_id in function_list:
        if data[children_id]['type'] == 'Expr':
            if data[data[children_id]['children'][0]]['type'] == 'Str':
                docstring = data[data[children_id]['children'][0]]['value']
    return docstring

In [6]:
# input: code data
# output: a tuple, (function_name, docstring, functions_call)

def find_function_info(data):
    
    ans = []
    full_list = find_function_children(data)
    
    for k, v in full_list.items():
        docstring = find_docstring(v[1], data)
        call_def = find_call_func(v[1], data)
        
        if docstring:
            ans.append((full_list[k][0]['value'], docstring, call_def))
        elif call_def:
            ans.append((full_list[k][0]['value'], '', call_def))
            
    return ans

In [7]:
# Read data and create pandas dataframe
function_info = []
for i, value in enumerate(all_data):
    func_doc = find_function_info(value)
    if func_doc:
        for tuples in func_doc:
            function_info.append([i, tuples[0], tuples[1], tuples[2]])

df = pd.DataFrame(np.array(function_info), columns=['data_id', 'function_name', 'docstring', 'func_call'])

In [8]:
df.head()

,data_id,function_name,docstring,func_call
0,1,__getattribute__,,__getattribute__
1,1,__setattr__,,"ref,__setattr__"
2,2,main,,"setup,closing,ZmqProxy,consume_in_thread,wait"
3,3,test_vpnservice_create,,"create_stubs,first,AndReturn,ReplayAll,vpnserv..."
4,3,test_vpnservices_get,,"create_stubs,AndReturn,ReplayAll,vpnservices_g..."


In [9]:
df[df.docstring != ''].head()

,data_id,function_name,docstring,func_call
47,8,ping,Handle ping requests,"iq,refresh,iq"
48,8,message,Proxy message from one user to another,"stanza,get,get,send_msg,endswith,warn,message,..."
57,8,presence,Presence information may be sent out from the ...,"stanza,warn,recv_presence,get,get,match_bare,s..."
60,8,roster,A roster is this account's list of contacts; i...,"iq,get,get"
63,8,push,Push roster changes to all clients that have r...,"requests,routes,iq,ignore"


In [10]:
df.shape

(1038, 4)

In [12]:
# export to pickle file for further training
df.to_pickle("./py100.pkl")